In [1]:
import joblib 
import numpy as np 
from matplotlib import pyplot as plt 
from uncertainties import ufloat, unumpy

from pathlib import Path
from tqdm import tqdm
from numpy import array
from core.utils import *
import atlas_mpl_style as ampl
#ampl.use_atlas_style(usetex=False)
import random
import mplhep as hep
hep.style.use("ATLAS")

In [25]:
#nominal_path = '/global/cfs/projectdirs/atlas/wys/QG_Calibration/uncertainty/SFs_pkls'
#nominal_path = Path(nominal_path)

reweighting_vars = ['jet_nTracks', 'GBDT_newScore']
nominal_keys = [reweighting_var + '_quark_reweighting_weights' for reweighting_var in reweighting_vars]
WPs = [0.5, 0.6, 0.7, 0.8]
partons=['Quark','Gluon']
label_ptrange = [500, 600, 800, 1000, 1200, 1500, 2000]
Map_var_title = {
    "jet_pt": "$p_{T}$",
    "jet_nTracks": "$n_{trk}$",
    "jet_trackBDT": "old BDT",
    "jet_eta": "$\eta$",
    "jet_trackC1": "$C_{1}$",
    "jet_trackWidth": "$w^{trk}$",
    "GBDT_newScore": "BDT"
}

In [9]:
pkl_path = '/global/cfs/projectdirs/atlas/wys/HEP_Repo/QG_Calibration/NewWorkflow/note_plots'
pkl_path = Path(pkl_path)
nominal_path = pkl_path / 'nominal' / 'plots' / 'ADE' / 'Extraction_Results'

def safe_array_divide_unumpy(numerator, denominator):
    if 0 in unumpy.nominal_values(denominator):
        _denominator_nominal_values = unumpy.nominal_values(denominator)
        _denominator_std_devs = unumpy.std_devs(denominator)
        zero_idx = np.where(_denominator_nominal_values==0)[0]
        _denominator_nominal_values[zero_idx] = np.inf
        _denominator_std_devs[zero_idx] = 0 
        _denominator = unumpy.uarray(_denominator_nominal_values, _denominator_std_devs)

        ratio = np.true_divide(numerator, _denominator) 
        # raise Warning(f"0 exists in the denominator for unumpy, check it!")
    else:
        ratio = np.true_divide(numerator, denominator)        
    return ratio

In [30]:
def Plot_WP(WP, var, output_path, period, reweighting_var,
            quark_effs, gluon_rejs, quark_effs_data, gluon_rejs_data,
            syst,if_save=True):
    
    SF_quark = safe_array_divide_unumpy(quark_effs_data, quark_effs)
    SF_gluon = safe_array_divide_unumpy(gluon_rejs_data, gluon_rejs)

    if if_save:
        bin_edges = np.array([500, 600, 800, 1000, 1200, 1500, 2000])
        bin_centers = 1/2 * (bin_edges[:-1] + bin_edges[1:])

        fig, ax0 = plt.subplots()
        if syst == 'nominal':
        #fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, gridspec_kw={'height_ratios': [4, 1], 'hspace': 0})
        # ax0.errorbar(x = bin_centers, y = unumpy.nominal_values(quark_effs), yerr = unumpy.std_devs(quark_effs), label = "Quark Efficiency, Extracted MC", color = "blue",linestyle='none', marker='^')
        # ax0.errorbar(x = bin_centers, y = unumpy.nominal_values(gluon_rejs), yerr = unumpy.std_devs(gluon_rejs), label = "Gluon Rejection, Extracted MC", color = "red",linestyle='none', marker='^')
        # ax0.errorbar(x = bin_centers, y = unumpy.nominal_values(quark_effs_data), yerr = unumpy.std_devs(quark_effs_data), label = "Quark Efficiency, Extracted Data", color= "blue", linestyle='none', marker= "o")
        # ax0.errorbar(x = bin_centers, y = unumpy.nominal_values(gluon_rejs_data), yerr = unumpy.std_devs(gluon_rejs_data), label = "Gluon Rejection, Extracted Data",color= "red", linestyle='none', marker= "o")
            hep.histplot(unumpy.nominal_values(quark_effs_data),bins=bin_edges,label='Quark Efficiency, Data',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(quark_effs_data),xerr=True,marker='.',color = "black")
            hep.histplot(unumpy.nominal_values(quark_effs),bins=bin_edges,label='Quark Efficiency, Pythia8',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(quark_effs),xerr=True,marker='^',color = "blue",markersize=5)

            hep.histplot(unumpy.nominal_values(gluon_rejs_data),bins=bin_edges,label='Gluon Rejection, Data',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(gluon_rejs_data),xerr=True,marker='*',color = "black",markersize=6)
            hep.histplot(unumpy.nominal_values(gluon_rejs),bins=bin_edges,label='Gluon Rejection, Pythia8',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(gluon_rejs),xerr=True,marker='v',color = "red",markersize=5)
        else:
            hep.histplot(unumpy.nominal_values(quark_effs),bins=bin_edges,label=f'Quark Efficiency,{syst}',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(quark_effs),xerr=True,marker='^',markersize=5)
            hep.histplot(unumpy.nominal_values(gluon_rejs),bins=bin_edges,label=f'Gluon Rejection,{syst}',ax=ax0,histtype='errorbar',yerr = unumpy.std_devs(gluon_rejs),xerr=True,marker='v',markersize=5)

        ax0.legend()
        #ax0.set_yticks(np.linspace(0, 1, 21))
        #ax0.set_xticks(bin_edges)
        ax0.set_ylim(0.3, 1.4)
        ax0.set_xlim(bin_edges[0], bin_edges[-1])
        ax0.set_ylabel("Efficiency or Rejection")
        ax0.set_xlabel('$p_{\mathrm{T}}$ [GeV]')


        #ax0.grid()
#        ax0.set_title(f"{var} for extracted q/g at {WP} WP")
        hep.atlas.label(label='Internal',data=True,ax=ax0,lumi=140)
        plt.text(600,1.18,Map_var_title[var])

        # ax1.errorbar(x = bin_centers, y = unumpy.nominal_values(SF_quark), yerr = unumpy.std_devs(SF_quark), linestyle='none', label = "quark SF", marker='.')
        # ax1.errorbar(x = bin_centers, y = unumpy.nominal_values(SF_gluon), yerr = unumpy.std_devs(SF_gluon), linestyle='none', label = "gluon SF", marker='.')
        # ax1.legend(fontsize = 'x-small')
        # ax1.set_ylim(0.7, 1.3)
        # ax1.set_xlim(bin_edges[0], bin_edges[-1])
        # #ampl.plot.set_xlabel(f"{Map_var_title[var]}")
        # #ax1.set_xticks(bin_edges)
        # ax1.hlines(y = 1, xmin = bin_edges[0], xmax = bin_edges[-1], color = 'gray', linestyle = '--')
        # ax1.set_ylabel("SFs")
        output_path=Path(output_path)
        output_path_new = output_path / period / "WPs" / reweighting_var / var
        if not output_path_new.exists():
            output_path_new.mkdir(parents = True)

        plt.show()
        fig.savefig( output_path_new/ f"{reweighting_var}_WP_{WP}.pdf")
        #plt.close()

    return SF_quark, SF_gluon

In [32]:
SFs = {}
Extraction_Results={}
syst_list=['nominal','hadronization/sherpa','hadronization/sherpalund','matrix_element/powhegpythia','parton_shower/herwigangle','parton_shower/herwigdipole']
#WP_cut_path = pkl_path/'nominal'/'plots'/'ADE'/'WP_cuts_pkls'/nominal_keys/'WP_cuts.pkl'
#WP_cut = joblib.load(WP_cut_path)
#WP_cut = dict.fromkeys(SF_label_vars)
output_path = 'note_plots'
for var in reweighting_vars:
    SFs[var] = {}
    WP_cut_path = pkl_path/'nominal'/'plots'/'ADE'/'WP_cuts_pkls'/ f'{var}_quark_reweighting_weights'/'WP_cuts.pkl'
    #WP_cut_path = '/global/cfs/projectdirs/atlas/wys/HEP_Repo/QG_Calibration/NewWorkflow/trained_lightGBM_new_hrzhao/nominal/plots/ADE/WP_cuts_pkls/jet_nTracks_quark_reweighting_weights/WP_cuts.pkl'
    WP_cut = joblib.load(WP_cut_path)
    #WP_cut = dict.fromkeys(var)
    #WP_cut[var] = dict.fromkeys(WPs)
    for syst in syst_list:
        SFs[var][syst] = {}
        syst_path= pkl_path / syst / 'plots' / 'ADE' / 'Extraction_Results'

        Extraction_Results[syst]= joblib.load(syst_path / f'{var}_Extraction_Results.pkl' )
        for l_pt in label_ptrange[:-1]:
            Extraction_var_pt =  Extraction_Results[syst][var][l_pt]


        for WP in WPs:
            #WP_cut[var][WP] = dict.fromkeys(label_ptrange[:-1])
            SFs[var][syst][WP] = {}
            quark_effs_at_pt = []
            gluon_rejs_at_pt = []
            quark_effs_data_at_pt = []
            gluon_rejs_data_at_pt = []
            for ii, l_pt in enumerate(label_ptrange[:-1]):
                extract_p_Quark_MC =  Extraction_Results[syst][var][l_pt]['extract_p_Quark_MC']
                extract_p_Gluon_MC =  Extraction_Results[syst][var][l_pt]['extract_p_Gluon_MC']
                extract_p_Quark_Data =  Extraction_Results[syst][var][l_pt]['extract_p_Quark_Data']
                extract_p_Gluon_Data =  Extraction_Results[syst][var][l_pt]['extract_p_Gluon_Data']

                #extract_p_Quark_cum_sum = np.cumsum(unumpy.nominal_values(extract_p_Quark_MC))
                #cut = np.where(extract_p_Quark_cum_sum >= WP)[0][0]+1
                cut = WP_cut[var][WP][l_pt]['idx']
                

                quark_effs_at_pt.append(np.sum(extract_p_Quark_MC[:cut])) 
                gluon_rejs_at_pt.append(np.sum(extract_p_Gluon_MC[cut:]))
                quark_effs_data_at_pt.append(np.sum(extract_p_Quark_Data[:cut]))
                gluon_rejs_data_at_pt.append(np.sum(extract_p_Gluon_Data[cut:]))

            SF_quark, SF_gluon = Plot_WP(WP = WP, var= var, output_path= output_path, 
                    period= 'ADE', reweighting_var = var,
                    quark_effs= quark_effs_at_pt, gluon_rejs = gluon_rejs_at_pt,
                    quark_effs_data=quark_effs_data_at_pt, gluon_rejs_data = gluon_rejs_data_at_pt,syst=syst)
            
        SFs[var][syst][WP]["Quark"] = SF_quark
        SFs[var][syst][WP]["Gluon"] = SF_gluon

KeyError: 'jet_nTracks'